In [1]:
import os
import obspy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, LSTM, Dense, Dropout
from tensorflow.keras.layers import TimeDistributed, ConvLSTM2D
from tensorflow.keras.utils import to_categorical
from pathlib import Path
%run helperfunctions.ipynb
#os.getcwd()

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
os.chdir('lse_challenge/data')

In [4]:
# Signal List
os.chdir('signal')
signals = []
signal_files = os.listdir()
signal_files = random.sample(signal_files, 500) #Need to expand for final model
for sigfile in signal_files: 
    tmp_trace = obspy.read(sigfile)
    tmp_trace = tmp_trace.normalize()
    cutlength = np.random.randint(16, 95) # Aligning with the [0.5 sec, 3 sec] range
    trace_mat = [tmp_trace[0].data[cutlength:1000+cutlength], tmp_trace[1].data[cutlength:1000+cutlength], tmp_trace[2].data[cutlength:1000+cutlength]]
    signals.append([trace_mat[0], cutlength, 1])
    signals.append([trace_mat[1], cutlength, 1])
    signals.append([trace_mat[2], cutlength, 1])
    
    # Negative Transformations
    neg_mat = rot_xyz(trace_mat, 0, -np.pi, 0)
    signals.append([neg_mat[0], cutlength, 1])
    signals.append([neg_mat[1], cutlength, 1])
    signals.append([neg_mat[2], cutlength, 1])
    
    # X, Y, and Z Rotations by Theta
    theta = np.random.uniform()*2*np.pi
    x_mat = rot_x(trace_mat, theta)
    signals.append([x_mat[0], cutlength, 1])
    signals.append([x_mat[1], cutlength, 1])
    signals.append([x_mat[2], cutlength, 1])
    y_mat = rot_y(trace_mat, theta)
    signals.append([y_mat[0], cutlength, 1])
    signals.append([y_mat[1], cutlength, 1])
    signals.append([y_mat[2], cutlength, 1])
    z_mat = rot_z(trace_mat, theta)
    signals.append([z_mat[0], cutlength, 1])
    signals.append([z_mat[1], cutlength, 1])
    signals.append([z_mat[2], cutlength, 1])    
    
    # General Rotation by alpha, beta, gamma
    alpha = np.random.uniform()*2*np.pi
    beta = np.random.uniform()*2*np.pi
    gamma = np.random.uniform()*2*np.pi
    rot_mat = rot_xyz(trace_mat, alpha, beta, gamma)
    signals.append([rot_mat[0], cutlength, 1])
    signals.append([rot_mat[1], cutlength, 1])
    signals.append([rot_mat[2], cutlength, 1])
    

# Noise List
os.chdir('../noise')
noise_files = os.listdir()
noise_files = random.sample(noise_files, 1500) #Need to expand for final model
for noisefile in noise_files: 
    tmp_trace = obspy.read(noisefile)
    tmp_trace = tmp_trace.normalize()
    cutlength = np.random.randint(16, 95) # Aligning with the [0.5 sec, 3 sec] range
    trace_mat = [tmp_trace[0].data[cutlength:1000+cutlength], tmp_trace[1].data[cutlength:1000+cutlength], tmp_trace[2].data[cutlength:1000+cutlength]]
    signals.append([trace_mat[0], cutlength, 0])
    signals.append([trace_mat[1], cutlength, 0])
    signals.append([trace_mat[2], cutlength, 0])
    
    # Negative Transformations
    neg_mat = rot_xyz(trace_mat, 0, -np.pi, 0)
    signals.append([neg_mat[0], cutlength, 0])
    signals.append([neg_mat[1], cutlength, 0])
    signals.append([neg_mat[2], cutlength, 0])
    
    # X, Y, and Z Rotations by Theta
    theta = np.random.uniform()*2*np.pi
    x_mat = rot_x(trace_mat, theta)
    signals.append([x_mat[0], cutlength, 0])
    signals.append([x_mat[1], cutlength, 0])
    signals.append([x_mat[2], cutlength, 0])
    y_mat = rot_y(trace_mat, theta)
    signals.append([y_mat[0], cutlength, 0])
    signals.append([y_mat[1], cutlength, 0])
    signals.append([y_mat[2], cutlength, 0])
    z_mat = rot_z(trace_mat, theta)
    signals.append([z_mat[0], cutlength, 0])
    signals.append([z_mat[1], cutlength, 0])
    signals.append([z_mat[2], cutlength, 0])
    
    # General Rotation by alpha, beta, gamma
    alpha = np.random.uniform()*2*np.pi
    beta = np.random.uniform()*2*np.pi
    gamma = np.random.uniform()*2*np.pi
    rot_mat = rot_xyz(trace_mat, alpha, beta, gamma)
    signals.append([rot_mat[0], cutlength, 0])
    signals.append([rot_mat[1], cutlength, 0])
    signals.append([rot_mat[2], cutlength, 0])
    
os.chdir('../')

sig_df = pd.DataFrame(signals, columns = ["trace", "cutlength", "signal"])
sig_df['p_arrival'] = 1000 - sig_df['cutlength']

## Training Neural Network for dectecting whether or not the trace has a signal

In [5]:
# For rolling averages
roll_short = 25
roll_long = 50

# For location segments
window_step = 15
window_size = 30

# Step size for CNN
n_steps_sig = 3
n_steps_loc = 3

In [6]:
# Features and Targets for Identifying the Entire Trace

sigfeatures = []
sigtargets = []

for index, slice_df in sig_df.iterrows():
    tr = slice_df["trace"]
    mag = abs(tr)
    signal = slice_df["signal"]
    p_arrival = slice_df["p_arrival"]
    d = {"trace": tr, "magnitude":mag}
    
    temp_df = pd.DataFrame(data = d)
    temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
    temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
    temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
    temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
    temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()
    
    temp_df.dropna(inplace = True)
    
    sigfeatures.append(temp_df.values)
    sigtargets.append(signal)

In [7]:
X_train_sig, X_test_sig, y_train_sig, y_test_sig = train_test_split(sigfeatures, sigtargets, test_size = 0.3)

In [8]:
X_train_sig = np.array(X_train_sig)
X_test_sig = np.array(X_test_sig)
y_train_sig = np.array(y_train_sig)
y_test_sig = np.array(y_test_sig)

In [9]:
n_timesteps, n_features, n_outputs = X_train_sig.shape[1], X_train_sig.shape[2], 2
n_length = int(n_timesteps/n_steps_sig)

In [ ]:
signalmodel = Sequential()
signalmodel.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps_sig, 1, n_length, n_features)))
signalmodel.add(Dropout(0.5))
signalmodel.add(Flatten())
signalmodel.add(Dense(100, activation='relu'))
signalmodel.add(Dense(2, activation='softmax'))
signalmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

signalmodel.summary()

In [10]:
X_train_sig1 = X_train_sig.reshape((X_train_sig.shape[0], n_steps_sig, 1, n_length, n_features))
X_test_sig1 = X_test_sig.reshape((X_test_sig.shape[0], n_steps_sig, 1, n_length, n_features))
y_train_sig1 = to_categorical(y_train_sig)
y_test_sig1 = to_categorical(y_test_sig)

In [ ]:
signalmodel.fit(X_train_sig1, y_train_sig1, epochs = 30)

In [ ]:
model_loss_sig, model_accuracy_sig = signalmodel.evaluate(X_test_sig1, y_test_sig1, verbose = 0)
print(f"Loss: {model_loss_sig}, Accuracy: {model_accuracy_sig}")

predictions_test_sig = signalmodel.predict(X_test_sig1)
predictions_train_sig = signalmodel.predict(X_train_sig1)

In [ ]:
class_test_sig = predictions_test_sig > 0.2
f1_score_test_sig = sklearn.metrics.f1_score(y_test_sig1, class_test_sig, average = 'micro')
print(f"Test F1 Score: {f1_score_test_sig}")

class_train_sig = predictions_train_sig > 0.2
f1_score_train_sig = sklearn.metrics.f1_score(y_train_sig1, class_train_sig, average = 'micro')
print(f"Train F1 Score: {f1_score_train_sig}")

In [ ]:
# ##@ TODO: Make compatible
# tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test_sig1, class_test_sig).ravel()
# (tn, fp, fn, tp)

In [11]:
# Exporting Model
os.getcwd()

'/Users/albertkong/grillochallenge/lse_challenge/data'

In [12]:
# Only run if we need to get to root directory
os.chdir('../../')

In [ ]:
# Exporting signal model
signalmodel.save('signalmodelCNN')

## Training Neural Network for dectecting where in the trace the p-wave starts

In [13]:
# Features and Targets for Identifying Windows of Trace
locfeatures = []
loctargets = []

for index, slice_df in sig_df.iterrows():
    
    signal = slice_df["signal"]
    
    if signal == 1:
        tr = slice_df["trace"]
        mag = abs(tr)

        p_arrival = slice_df["p_arrival"]
        d = {"trace":tr, "magnitude":mag}

        temp_df = pd.DataFrame(data = d)
        temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
        temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
        temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
        temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
        temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()

        temp_df.dropna(inplace = True)

        start_ind = 0
        end_ind = start_ind + window_size

        while end_ind < (1000 - roll_long):
            trwindow = temp_df["trace"].iloc[start_ind:end_ind]
            magwindow = temp_df["magnitude"].iloc[start_ind:end_ind]
            ravwindow = temp_df["RAV"].iloc[start_ind:end_ind]
            stvwindow = temp_df["STV"].iloc[start_ind:end_ind]
            ltvwindow = temp_df["LTV"].iloc[start_ind:end_ind]

            window_data = {"trace": trwindow, "magnitude": magwindow,
                        "RAV": ravwindow, "STV": stvwindow, "LTV": ltvwindow}
            window_df = pd.DataFrame(data = window_data)

            locfeatures.append(window_df.values)

            if ((p_arrival - roll_long + 1) >= start_ind) and ((p_arrival - roll_long + 1) <= end_ind):
                loctargets.append(1)
            else:
                loctargets.append(0)

            start_ind += window_step
            end_ind = start_ind + window_size

In [14]:
# For Windows of Trace
X_train_loc, X_test_loc, y_train_loc, y_test_loc = train_test_split(locfeatures, loctargets, test_size = 0.3)

In [15]:
X_train_loc = np.array(X_train_loc)
X_test_loc = np.array(X_test_loc)
y_train_loc = np.array(y_train_loc)
y_test_loc = np.array(y_test_loc)

In [16]:
n_timesteps, n_features, n_outputs = X_train_loc.shape[1], X_train_loc.shape[2], 2
n_length = int(n_timesteps/n_steps_loc)

In [ ]:
locmodel = Sequential()
locmodel.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps_loc, 1, n_length, n_features)))
locmodel.add(Dropout(0.5))
locmodel.add(Flatten())
locmodel.add(Dense(100, activation='relu'))
locmodel.add(Dense(2, activation='softmax'))
locmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

signalmodel.summary()

In [17]:
X_train_loc1 = X_train_loc.reshape((X_train_loc.shape[0], n_steps_loc, 1, n_length, n_features))
X_test_loc1 = X_test_loc.reshape((X_test_loc.shape[0], n_steps_loc, 1, n_length, n_features))
y_train_loc1 = to_categorical(y_train_loc)
y_test_loc1 = to_categorical(y_test_loc)

In [ ]:
locmodel.fit(X_train_loc1, y_train_loc1, epochs = 15)

In [22]:
model_loss_loc, model_accuracy_loc = locmodel.evaluate(X_test_loc1, y_test_loc1, verbose = 0)
print(f"Loss: {model_loss_loc}, Accuracy: {model_accuracy_loc}")

predictions_test_loc = locmodel.predict(X_test_loc1)
predictions_train_loc = locmodel.predict(X_train_loc1)

2022-03-13 09:12:25.474635: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-13 09:12:25.644061: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Loss: 0.10070851445198059, Accuracy: 0.9813799262046814


2022-03-13 09:13:21.202877: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [32]:
class_test_loc = predictions_test_loc > 0.25
f1_score_test_loc = sklearn.metrics.f1_score(y_test_loc1, class_test_loc, average='micro')
print(f"Test F1 Score: {f1_score_test_loc}")

class_train_loc = predictions_train_loc > 0.25
f1_score_train_loc = sklearn.metrics.f1_score(y_train_loc1, class_train_loc, average='micro')
print(f"Train F1 Score: {f1_score_train_loc}")

Test F1 Score: 0.982568697729988
Train F1 Score: 0.9829134664618535


In [33]:
# ##@ TODO: Make compatible
# tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test_loc[:,1], class_test_loc[:,1]).ravel()
# (tn, fp, fn, tp)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
# Exporting location model
locmodel.save('locmodelCNN')

## Spot checking results

In [18]:
loaded_signalmodel = load_model('signalmodelCNN1')

2022-03-13 09:11:15.433246: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-13 09:11:15.433709: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [19]:
loaded_locmodel = load_model('locmodelCNN1')

In [ ]:
slice_df = sig_df.iloc[29]

In [ ]:
sigfeatures = []
sigtargets = []
locfeatures = []
loctargets = []

tr = slice_df["trace"]
mag = abs(tr)
signal = slice_df["signal"]
p_arrival = slice_df["p_arrival"]
d = {"trace": tr, "magnitude":mag}
temp_df = pd.DataFrame(data = d)
temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()
    
temp_df.dropna(inplace = True)
sigfeatures.append(temp_df.values)
sigtargets.append(signal)

sigfeatures = np.array(sigfeatures)
n_timesteps, n_features, n_outputs = sigfeatures.shape[1], sigfeatures.shape[2], 2
n_length = int(n_timesteps/n_steps_sig)

sigfeatures1 = sigfeatures.reshape((sigfeatures.shape[0], n_steps_sig, 1, n_length, n_features))

In [ ]:
loaded_signalmodel.predict(sigfeatures1)[0][1]

In [ ]:
sigtargets

In [ ]:
start_ind = 0
end_ind = start_ind + window_size
    
while end_ind < (1000 - roll_long):
    magwindow = temp_df["magnitude"].iloc[start_ind:end_ind]
    trwindow = temp_df["trace"].iloc[start_ind:end_ind]
    ravwindow = temp_df["RAV"].iloc[start_ind:end_ind]
    stvwindow = temp_df["STV"].iloc[start_ind:end_ind]
    ltvwindow = temp_df["LTV"].iloc[start_ind:end_ind]

    window_data = {"trace":trwindow, "magnitude": magwindow,
                    "RAV": ravwindow, "STV": stvwindow, "LTV": ltvwindow}
        
    window_df = pd.DataFrame(data = window_data)

    locfeatures.append(window_df.values)

    if ((p_arrival-roll_long) >= start_ind) and ((p_arrival-roll_long) <= end_ind):
        loctargets.append(1)
    else:
        loctargets.append(0)

    start_ind += window_step
    end_ind = start_ind + window_size
    

locfeatures = np.array(locfeatures)
n_timesteps, n_features, n_outputs = locfeatures.shape[1], locfeatures.shape[2], 2
n_length = int(n_timesteps/n_steps_loc)

locfeatures1 = locfeatures.reshape((locfeatures.shape[0], n_steps_loc, 1, n_length, n_features))

In [ ]:
loaded_locmodel.predict(locfeatures1)[:,1]

In [ ]:
loctargets